In [2]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

import os
import random

from sentence_transformers import SentenceTransformer

/home/amir/miniconda3/envs/nlp/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
model = SentenceTransformer("all-MiniLM-L6-v2").to(device).eval()

/home/amir/miniconda3/envs/nlp/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Walk into directoies to find images and convert them to vectors
txt_paths = []
sentences = []
for root, dirs, files in os.walk('data/'):
    for file in files:
        if file.endswith((".txt")):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read().replace('\n', '')
                sentences.append(text)


            txt_paths.append(file_path)
sentences = list(set(sentences))

In [6]:
embeddings = model.encode(sentences)
embeddings.shape

(6391, 384)

### Similarity Search

In [21]:
idx = random.randint(0, len(sentences))

query = embeddings[idx]
query_sentence

'يك  كارشناس مسايل رژيم صهيونيستي : توافق احتمالي ايران و آمريكا  موجب نگراني اسرائيل است تهران - "مئير جاودانفر" با عنوان كارشناس مسايل اسرائيل درباره نگراني هاي رژيم صهيونيستي از توافق بين ايران و آمريكا گفت : جزئيات توافق نهايي بين ايران و آمريكا راجع به برنامه اتمي ايران - موجب نگراني اسرائيل است  به گزارش گروه اخبار ايران در رسانه هاي جهان - جاودانفر در گفت وگوبا تلويزيون فارسي بي بي سي در پاسخ به اين سووال كه "نزديك شدن احتمالي تهران و واشنگتن تا چه حد ممكن است روابط دو دولت اسرائيل وآمريكا را سردتر كند؟" بيان داشت : بستگي به اين دارد كه توافقنامه نهايي بين ايران و آمريكا چه شكلي داشته باشد: اگر در حد مكالمه و صحبت با همديگر باشد- اين مساله نتانياهو (نخست وزير رژيم صهيونيستي ) و دولت اسرائيل را زياد نگران نمي كند: آن چه كه واقعا باعت ايجاد نگراني دراسرائيل است جزئيات يك توافق نهايي بين ايران و آمريكا راجع به برنامه اتمي ايران خواهد بود "اگر دولت اسرائيل با اين جزئيات موافقت نكند و باراك اوباما آن توافقنامه را امضا كند- باعت ضربه زدن به روابط بين اسرائيل و آمريكاخواهد شد "جاودانفر

In [22]:
similarities = model.similarity(query, embeddings)
similarities

tensor([[0.9022, 0.8144, 0.9239,  ..., 0.9023, 0.8773, 0.8253]])

In [23]:
values, indices = similarities.sort(descending=True)

In [25]:
neighbors = [sentences[i] for i in indices[0][:10]]
df = pd.DataFrame(neighbors, columns=['Neigbors'])
df

,Neigbors
0,يك كارشناس مسايل رژيم صهيونيستي : توافق احتما...
1,جان كري به سوريه هشدار داد تهران -ايرنا -جان ك...
2,هيگل حمله رژيم صهيونيستي به ايران را به تصميم...
3,المانيتور:كنگره آمريكا به نوع ديگري از تحري...
4,موسسه امريكايي :تركيه مي تواند به سوريه حمله ...
5,پاتريك كلاوسون: فوايد معامله هسته اي براي تهرا...
6,ادعاي گلوله باران مناطق مرزي كردستان عراق از ...
7,كارشناسان غربي :شكست مذاكرات :فرصتي براي پيشر...
8,تارنماي رژيم صهيونيستي : تل آويو براي مقابله ...
9,تارنماي رژيم صهيونيستي : تل آويو براي مقابله ...


In [26]:
with pd.option_context('display.max_colwidth', None):
  display(df)

Neigbors
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   يك  كارشناس مسايل رژيم صهيونيستي : توافق احتمالي ايران و آمريكا  موجب نگراني اسرائيل است تهران - "مئير جاودانفر" با عنوان كارشناس مسايل اسرائيل درباره نگراني هاي رژيم صهيونيستي از توافق بين ايران و آمريكا گفت : جزئيات توافق نهايي بين ايران و آمريكا راجع به برنامه اتمي ايران - موجب نگراني اسرائيل است  به گزارش گروه اخبار ايران در رسانه هاي جهان - جاودانفر در گفت وگوبا تلويزيون فارسي بي بي سي در پاسخ به اين سووال كه "نزديك شدن احتمالي تهران و واشنگتن تا چه حد ممكن است روابط دو دولت اسرائيل وآمريكا را سردتر كند؟" بيان داشت : بستگي به اين دارد كه توافقنامه نهايي بين ايران و آمريكا چه شكلي داشته باشد: اگر در حد مكالمه و صحبت با همديگر باشد- اين مساله نتانياهو (نخست وزير رژيم صهيونيستي ) و دولت اسرائيل را زياد نگران نمي كند: آن چه كه واقعا باعت ايجاد نگراني دراسرائيل است جزئيات يك توافق نهايي بين ايران و آمريكا راجع به برنامه اتمي ايران خواهد بود "اگر دولت اسرائيل با اين جزئيات موافقت نكند و باراك اوباما آن توافقنامه را امضا كند- باعت ضربه زدن به روابط بين اسرائيل و آمريكاخواهد شد "جاودانفر درباره اين پرسش كه "نتانياهو به ديپلماسي صلح طلبانه حسن روحاني (رييس جمهوري ايران ) نگاه منفي دارد و از اصطلاحاتي همچون شيرين زباني اغواگرايانه استفاده كرده آيا ممكن است كه اوباما موفق شود دراين بدبيني دولت اسراييل و به طور مشخص نتانيا